In [ ]:
!pip uninstall keras -y
!pip install keras==2.3.1

In [ ]:
import keras 
from keras.models import load_model
from keras.layers import (Input, Concatenate, Dense, Lambda, Reshape, Dropout, Flatten, 
                          UpSampling2D, Activation, Conv2D, Conv2DTranspose, BatchNormalization, AveragePooling2D, ZeroPadding2D, RepeatVector)
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Deconv2D
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.utils import to_categorical
from sklearn.utils import shuffle
import matplotlib.cm as cm
from matplotlib.colors import LogNorm, Normalize
from keras.losses import mse, binary_crossentropy
import tensorflow as tf
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import numpy as np
import numpy
import h5py

In [ ]:
data = h5py.File('../input/jets-dataset/PYTHIA_JETS.hdf5', 'r')
JETS = numpy.array(data["image"][:])

data2 = h5py.File('../input/jets-dataset/PYTHIA_JETS.hdf5', 'r')
JETS_label = numpy.array(data2["signal"][:])

JETS, JETS_label = shuffle(JETS, JETS_label)

JETS_label = JETS_label[0:870000].reshape(870000,1)
JETS = JETS[0:870000].reshape(870000,25,25,1)

JETS[JETS <1e-3] = 0

In [ ]:
classifier = load_model('../input/classifier2/11X11_CNN_LR3.h5')
selected_layers = ['leaky_re_lu_1','leaky_re_lu_2']
selected_weights = [1, 1]

In [ ]:
classifier.summary()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(JETS, JETS_label, train_size=0.9)

X_train = X_train.astype('float32')/100.
X_test = X_test.astype('float32')/100.

Y_train = to_categorical(Y_train, num_classes = 2)
Y_test = to_categorical(Y_test, num_classes = 2)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
latent_size = 12
batch_size = 100
c_space = Y_train.shape[1]
n_epochs = 12

In [ ]:
image = Input(shape = (25,25,1))
label = Input(shape = (Y_train.shape[1],))

transform = RepeatVector(625)(label)
label_2D = Reshape((25,25, Y_train.shape[1]))(transform)
inputs = Concatenate()([image, label_2D])


x = Conv2D(32, kernel_size = (3,3), padding = 'same')(inputs)
x = LeakyReLU()(x)

x = Conv2D(64, kernel_size = (3,3), padding = 'same', strides = 2)(x)
x = LeakyReLU()(x)

x = Conv2D(128, kernel_size = (3,3), padding = 'same', strides = 2)(x)
x = LeakyReLU()(x)

encoder_output = Flatten()(x)

mu = Dense(latent_size, activation = 'linear')(encoder_output)
sigma = Dense(latent_size, activation = 'linear')(encoder_output)

def sampling(args):
    mu, sigma = args
    eps = K.random_normal(shape=(batch_size, latent_size), mean=0., stddev=1.0)
    return mu + K.exp(sigma/2.) * eps

latent_space = Lambda(sampling, output_shape=(latent_size,))([mu, sigma])
combined_space = Concatenate()([latent_space, label])

decoder1 = Dense((7*7*128))
decoder2 = Reshape((7,7,128))

decoder3 = Deconv2D(64, kernel_size = (3, 3), padding='same', strides = 2)
decoder4 = LeakyReLU()

decoder5 = Deconv2D(32, kernel_size = (3, 3), padding='same', strides = 2)
decoder6 = LeakyReLU()

decoder7 = Conv2D(1, kernel_size = (4,4), padding='valid')

decoder_out = Activation('relu')

In [ ]:
encoder = Model(inputs = [image, label], outputs = latent_space)
encoder.summary()

In [ ]:
inputs = Input(shape=(latent_size + c_space, ))

d = decoder1(inputs)
d = decoder2(d)
d = decoder3(d)
d = decoder4(d)
d = decoder5(d)
d = decoder6(d)
d = decoder7(d)
d = decoder_out(d)
decoder = Model(inputs, d)
decoder.summary()

In [ ]:
d_c = decoder1(combined_space)
d_c = decoder2(d_c)
d_c = decoder3(d_c)
d_c = decoder4(d_c)
d_c = decoder5(d_c)
d_c = decoder6(d_c)
d_c = decoder7(d_c)
dc_out = decoder_out(d_c)

VAE = Model(inputs = [image, label], outputs = dc_out)


def DFC_loss(x_in, x_out):
    kl_loss = 0.5 * K.sum(K.exp(sigma) + K.square(mu) - 1. - sigma, axis=-1)
    return K.mean((kl_loss) + perceptual_loss(x_in,x_out) + (bce_loss(x_in, x_out)))

def bce_loss(x_in, x_out):
    x_in = K.reshape(x_in, shape=(batch_size, 625))
    x_out = K.reshape(x_out, shape=(batch_size, 625))
    
    return binary_crossentropy(x_in, x_out)

def perceptual_loss(x_in, x_out):
    
    x_in = K.reshape(x_in, shape=(batch_size, 25,25,1))
    x_out = K.reshape(x_out, shape=(batch_size, 25,25,1))
    
    conv_outputs = [classifier.get_layer(l).output for l in selected_layers]
    
    activation = Model(classifier.input, conv_outputs)

    h1_list = activation(x_in)
    h2_list = activation(x_out)
    
    rc_loss = 0.0
    
    for h1, h2, weight in zip(h1_list, h2_list, selected_weights):
        h1 = K.batch_flatten(h1)
        h2 = K.batch_flatten(h2)
        rc_loss = rc_loss + weight * K.sum(K.square(h1 - h2), axis=-1)
    
    return rc_loss
VAE.compile(optimizer = "adam", loss = DFC_loss, metrics = [perceptual_loss])
VAE.summary()

In [ ]:
VAE_hist = VAE.fit([X_train, Y_train], X_train, verbose = 1, batch_size=batch_size, epochs=n_epochs,
                            validation_data = ([X_test, Y_test], X_test), callbacks = [EarlyStopping(patience = 5)])

In [ ]:
VAE.save('Model4paper_VAE.h5')
decoder.save('Model4paper_decoder.h5')
encoder.save('Model4paper_encoder.h5')